In [1]:
from swissisoform.genome import GenomeHandler
from swissisoform.visualize import GenomeVisualizer
from swissisoform.isoform import AlternativeIsoform
from swissisoform.mutations import MutationHandler
from swissisoform.utils import analyze_mutations
from typing import Optional, List
import pandas as pd
import os

In [2]:
genome = GenomeHandler(
    "../data/genome_data/hg38.fa", "../data/genome_data/hg38.ncbiRefSeq.gtf"
)

alt_isoforms = AlternativeIsoform()
alt_isoforms.load_bed(
    "../data/ribosome_profiling/RiboTISHV6_MD2025_AnnoToTruncation_exonintersect.bed"
)

mutation_handler = MutationHandler()

In [3]:
gene_name = "NAXE"
print(f"Processing gene: {gene_name}")

os.makedirs(f"./{gene_name}", exist_ok=True)

# Get alternative isoform features
print("Getting alternative features...")
alt_features = alt_isoforms.get_visualization_features(gene_name)

if alt_features.empty:
    print("No alternative features found")
else:
    print(f"Found {len(alt_features)} alternative features")
    print("\nAlternative Features:")
    print(alt_features)

Processing gene: NAXE
Getting alternative features...
Found 1 alternative features

Alternative Features:
    chromosome      source       feature_type      start        end  score  \
309       chr1  truncation  alternative_start  156591805  156591957      0   

    strand frame             gene_id           transcript_id gene_name  \
309      +     .  ENSG00000163382.11  ENSG00000163382.11_alt      NAXE   

    start_codon  
309         AUG  


In [4]:
print("Getting transcript information...")
transcript_info = genome.get_transcript_ids(gene_name)

if transcript_info.empty:
    print("No transcript info found")
else:
    print(f"Found {len(transcript_info)} transcripts")
    print("\nTranscript Information:")
    print(transcript_info)

Getting transcript information...
Note: Filtered out 1 transcript(s) on non-standard chromosomes
Non-standard chromosomes: {'chr1_MU273335v1_fix'}
Found 1 transcripts

Transcript Information:
         transcript_id chromosome      start        end strand
4571171  NM_144772.3_2       chr1  156591776  156594299      +


In [5]:
mutations_unfiltered = await analyze_mutations(
    gene_name=gene_name,
    mutation_handler=mutation_handler,
    alt_features=alt_features,
    sources=["clinvar"],
)

impact_types = {
    "clinvar": ["missense variant", "nonsense variant", "frameshift variant"],
}

mutations_filtered = await analyze_mutations(
    gene_name=gene_name,
    mutation_handler=mutation_handler,
    alt_features=alt_features,
    sources=["clinvar"],
    impact_types=impact_types,
)

Fetching mutations from sources: clinvar...
Fetching mutations from sources: clinvar...


/lab/barcheese01/mdiberna/swissisoform/src/swissisoform/mutations.py:390: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')



Analyzing mutations in alternative features:
  ├─ Feature 310 (156591805-156591957): 17 mutations

Total mutations in all features: 17
Found 17 mutations in truncation regions

Mutation Analysis:
Impact types: {'missense variant': 11, 'synonymous variant': 3, 'nonsense variant': 2, 'inframe deletion': 1}
Clinical significance: {'Uncertain significance': 10, 'Likely benign': 4, 'Benign': 1, 'Likely pathogenic': 1, 'Pathogenic': 1}
Truncation regions: ['156591805-156591957']
Fetching mutations from sources: clinvar...
Fetching mutations from sources: clinvar...

Analyzing mutations in alternative features:
  ├─ Feature 310 (156591805-156591957): 17 mutations

Total mutations in all features: 17
Filtering for impact types by source:
  - clinvar: missense variant, nonsense variant, frameshift variant
Found 13 mutations in truncation regions

Mutation Analysis:
Impact types: {'missense variant': 11, 'nonsense variant': 2}
Clinical significance: {'Uncertain significance': 9, 'Likely benign'

In [6]:
visualizer = GenomeVisualizer(genome)

# For each transcript, create visualizations
for _, transcript in transcript_info.iterrows():
    transcript_id = transcript["transcript_id"]
    print(f"\nVisualizing transcript: {transcript_id}")

    # Define the directory where plots will be saved
    output_dir = f"./{gene_name}/"  # Assuming you already created this directory

    # Create visualization with unfiltered mutations
    if mutations_unfiltered is not None:
        visualizer.visualize_transcript(
            gene_name=gene_name,
            transcript_id=transcript_id,
            alt_features=alt_features,
            mutations_df=mutations_unfiltered,
            output_file=f"{output_dir}{transcript_id}_unfiltered.png",
        )

    # Create visualization with filtered mutations
    if mutations_filtered is not None:
        visualizer.visualize_transcript(
            gene_name=gene_name,
            transcript_id=transcript_id,
            alt_features=alt_features,
            mutations_df=mutations_filtered,
            output_file=f"{output_dir}{transcript_id}_filtered.png",
        )

        # Create zoomed view for filtered mutations
        visualizer.visualize_transcript_zoomed(
            gene_name=gene_name,
            transcript_id=transcript_id,
            alt_features=alt_features,
            mutations_df=mutations_filtered,
            output_file=f"{output_dir}{transcript_id}_filtered_zoom.png",
            padding=100,
        )


Visualizing transcript: NM_144772.3_2
